<a href="https://colab.research.google.com/github/pantso251/DSFSE_Portfolio/blob/main/clean_coords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [ ]:
#mount drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
#read the merged file
df = pd.read_csv("/content/merged_dataset_v2.csv")

In [ ]:
df.tail(5)

,Unnamed: 0,Beach_Code,Original_Beach_Name,API_Beach_Name,rating,user_ratings_total,lat,lon,Date of collection,Period,length,Trash amount,Beach was cleaned,The last time that beach has been cleaned,The last time that was cleaned from another group,further from the reference point above 100 meters,Trovato un animale abbandonato o morto,Numero di animali abbandonati o morti
184,205,120,Tombolo della Feniglia,Feniglia,4.4,3158,4.241862e-02,1.124465e-02,30/12/2025,Winter 2025-2026,0,69.0,True,06/03/2025,False,True,False,NaN
185,207,30,trabucco,Spiaggia libera Le Dune,4.0,1541,4.398257e-07,1.013512e-07,10/07/2025,Summer 2025,"64,16",653.0,True,29/05/2025,False,False,False,NaN
186,208,30,trabucco,Spiaggia libera Le Dune,4.0,1541,4.398257e-07,1.013512e-07,14/11/2025,Autumn 2025,"64,16",214.0,True,14/11/2025,False,False,False,0.0
187,209,87,Vergine Maria,Spiaggia Vergine Maria,4.0,2494,3.816626e+01,1.336852e+01,28/06/2025,Summer 2025,"140,01",148.0,True,28/06/2025,False,True,False,NaN
188,210,87,Vergine Maria,Spiaggia Vergine Maria,4.0,2494,3.816626e+01,1.336852e+01,06/12/2025,Winter 2025-2026,"140,01",1685.0,False,28/06/2025,False,False,False,NaN


In [ ]:
# First, convert to float
df[['lat','lon']] = df[['lat','lon']].astype(float)

#Check the range that
print("Original min/max:")
print(f"Lat: {df['lat'].min():.10f} to {df['lat'].max():.10f}")
print(f"Lon: {df['lon'].min():.10f} to {df['lon'].max():.10f}")

Original min/max:
Lat: 0.0000003744 to 45.3467824800
Lon: 0.0000001000 to 98.9480316600


As it can be seen the range of the coordinates it's very big and in the code block below we are trying to make the format consistent

In [ ]:
# Very small scientific notation (e-07, e-08)
# These need ×100,000,000 to get to ~37-44°
mask_pattern1_lat = df['lat'] < 1e-06  # Less than 0.000001
mask_pattern1_lon = df['lon'] < 1e-06  # Less than 0.000001
df.loc[mask_pattern1_lat, 'lat'] *= 100000000
df.loc[mask_pattern1_lon, 'lon'] *= 100000000

# Small decimals (0.01 to 0.1 range)
# These need ×1000 to get to ~10-40°
mask_pattern2_lat = df['lat'].between(0.001, 1)
mask_pattern2_lon = df['lon'].between(0.001, 1)
df.loc[mask_pattern2_lat, 'lat'] *= 1000
df.loc[mask_pattern2_lon, 'lon'] *= 1000

# Medium small (3-10 range) - latitude only
# These need ×10 to get to ~30-100°
mask_pattern3_lat = df['lat'].between(3, 10)
df.loc[mask_pattern3_lat, 'lat'] *= 10

# Longitudes that are too large (> 19°)
mask_pattern4_lon = df['lon'] > 19
df.loc[mask_pattern4_lon, 'lon'] /= 10

# Check if any latitudes are still too small (< 36°)
mask_pattern5_lat = df['lat'] < 36
df.loc[mask_pattern5_lat, 'lat'] *= 10

print("\nAfter first pass:")
print(f"Lat: {df['lat'].min():.6f} to {df['lat'].max():.6f}")
print(f"Lon: {df['lon'].min():.6f} to {df['lon'].max():.6f}")


After first pass:
Lat: 36.719600 to 45.346782
Lon: 1.387115 to 17.968869


In [ ]:
# Fix any remaining outliers
df.loc[df['lat'] < 36, 'lat'] = df.loc[df['lat'] < 36, 'lat'] * 10
df.loc[df['lat'] > 47, 'lat'] = df.loc[df['lat'] > 47, 'lat'] / 10
df.loc[df['lon'] < 6, 'lon'] = df.loc[df['lon'] < 6, 'lon'] * 10
df.loc[df['lon'] > 19, 'lon'] = df.loc[df['lon'] > 19, 'lon'] / 10


#Check the final ranges
print("\nFinal ranges:")
print(f"Lat: {df['lat'].min():.6f} to {df['lat'].max():.6f}")
print(f"Lon: {df['lon'].min():.6f} to {df['lon'].max():.6f}")

print("\nChecking if all coordinates are in reasonable Italian ranges...")
lat_in_range = ((df['lat'] >= 36) & (df['lat'] <= 47)).all()
lon_in_range = ((df['lon'] >= 6) & (df['lon'] <= 19)).all()
print(f"All latitudes in 36-47° range: {lat_in_range}")
print(f"All longitudes in 6-19° range: {lon_in_range}")


Final ranges:
Lat: 36.719600 to 45.346782
Lon: 8.408904 to 17.968869

Checking if all coordinates are in reasonable Italian ranges...
All latitudes in 36-47° range: True
All longitudes in 6-19° range: True


In [ ]:
df.head(5)

,Unnamed: 0,Beach_Code,Original_Beach_Name,API_Beach_Name,rating,user_ratings_total,lat,lon,Date of collection,Period,length,Trash amount,Beach was cleaned,The last time that beach has been cleaned,The last time that was cleaned from another group,further from the reference point above 100 meters,Trovato un animale abbandonato o morto,Numero di animali abbandonati o morti
0,0,40,Aciddara,Spiaggia Aciddara,3.5,357,38.080366,13.537345,12/05/2025,Spring 2025,"149,53",1794.0,True,NaN,False,False,True,2.0
1,1,40,Aciddara,Spiaggia Aciddara,3.5,357,38.080366,13.537345,13/07/2025,Summer 2025,"149,53",236.0,True,12/04/2024,False,False,False,NaN
2,2,40,Aciddara,Spiaggia Aciddara,3.5,357,38.080366,13.537345,30/12/2025,Winter 2025-2026,"149,53",552.0,False,13/07/2025,False,True,False,0.0
3,3,149,Bagni della Regina Giovanna - Sorrento (Na),Capo di Sorrento,4.7,153,40.633559,14.351224,14/12/2025,Winter 2025-2026,"9,39",738.0,True,NaN,False,False,True,1.0
4,5,123,Boschetto Steccato di Cutro,Spiaggia Libera Steccato di Cutro,4.4,279,38.934628,16.931957,09/07/2025,Summer 2025,"35,21",418.0,True,18/07/2025,False,False,False,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 18 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Unnamed: 0                                          189 non-null    int64  
 1   Beach_Code                                          189 non-null    int64  
 2   Original_Beach_Name                                 189 non-null    object 
 3   API_Beach_Name                                      189 non-null    object 
 4   rating                                              189 non-null    float64
 5   user_ratings_total                                  189 non-null    int64  
 6   lat                                                 189 non-null    float64
 7   lon                                                 189 non-null    float64
 8   Date of collection                                  189 non-null    object 
 9  

In [ ]:
df_clean = df.drop(columns = "Unnamed: 0")

In [ ]:
## if you want to save it
df_clean.to_csv('cleaned_data.csv', index=False)

**What went good:**

This jupyter notebook successfully read the coordinates of the file, bring into a consistent format and creates a new file csv file.

---


**Want went bad:**

The coordinates of this file merged_dataset_v2 are manually created for some beaches that had a bad location. The result of that action was to bring into a inconsistent format

---
**What could be done better:**

During the manually selection of the coordinates to keep the same format with rest in order to avoid this jupyter notebook file.

####Option 2 Below but it has to be integrated.


In [ ]:
'''
# Read the corrected beaches file
coords = pd.read_csv("beaches_corrected.csv") # Changed from beaches_final_clean.csv
coords.head()

# Convert and scale ONLY API_lat/API_lng (the reliable ones)
coords['API_lat'] = pd.to_numeric(coords['API_lat'], errors='coerce') / 1e8
coords['API_lng'] = pd.to_numeric(coords['API_lng'], errors='coerce') / 1e8

# Fix rating (divide by 10 if >= 10)
coords['rating'] = pd.to_numeric(coords['rating'], errors='coerce')
coords['rating'] = np.where(coords['rating'] >= 10, coords['rating'] / 10, coords['rating'])

# Keep ONLY the columns we need - discard all old coordinate columns
coords_clean = coords[[
    'Beach_Code',
    'Original_Beach_Name',
    'API_Beach_Name',
    'rating',
    'user_ratings_total',
    'API_lat',
    'API_lng'
]].copy()

# Filter: keep only valid Italian coordinates
coords_clean = coords_clean[
    (coords_clean['API_lat'].between(35, 48)) &
    (coords_clean['API_lng'].between(6, 20)) &
    coords_clean['API_lat'].notna() &
    coords_clean['API_lng'].notna()
]

print(f"Clean beaches with valid coordinates: {len(coords_clean)}")
coords_clean.to_csv("beaches_final_clean.csv", index=False)
'''

'\n# Read the corrected beaches file\ncoords = pd.read_csv("beaches_corrected.csv") # Changed from beaches_final_clean.csv\ncoords.head()\n\n# Convert and scale ONLY API_lat/API_lng (the reliable ones)\ncoords[\'API_lat\'] = pd.to_numeric(coords[\'API_lat\'], errors=\'coerce\') / 1e8\ncoords[\'API_lng\'] = pd.to_numeric(coords[\'API_lng\'], errors=\'coerce\') / 1e8\n\n# Fix rating (divide by 10 if >= 10)\ncoords[\'rating\'] = pd.to_numeric(coords[\'rating\'], errors=\'coerce\')\ncoords[\'rating\'] = np.where(coords[\'rating\'] >= 10, coords[\'rating\'] / 10, coords[\'rating\'])\n\n# Keep ONLY the columns we need - discard all old coordinate columns\ncoords_clean = coords[[\n    \'Beach_Code\',\n    \'Original_Beach_Name\',\n    \'API_Beach_Name\',\n    \'rating\',\n    \'user_ratings_total\',\n    \'API_lat\',\n    \'API_lng\'\n]].copy()\n\n# Filter: keep only valid Italian coordinates\ncoords_clean = coords_clean[\n    (coords_clean[\'API_lat\'].between(35, 48)) &\n    (coords_clean[\

In [ ]:
'''

# Read file
coords = pd.read_csv("beaches_corrected.csv")

# Check if lat/lon columns already exist in decimal format
if 'lat' in coords.columns and 'lon' in coords.columns:
    print("✅ lat/lon columns already exist!")

    # Just clean them
    coords['lat'] = pd.to_numeric(coords['lat'], errors='coerce')
    coords['lon'] = pd.to_numeric(coords['lon'], errors='coerce')

else:
    # Create from Search columns
    coords['lat'] = pd.to_numeric(coords['Search_lat'], errors='coerce')
    coords['lon'] = pd.to_numeric(coords['Search_lng'], errors='coerce')

    # Check if they look like big integers that need scaling
    if coords['lat'].max() > 1000:
        print("Scaling coordinates...")
        coords['lat'] = coords['lat'] / 1e8
        coords['lon'] = coords['lon'] / 1e8

# Fix rating
coords['rating'] = pd.to_numeric(coords['rating'], errors='coerce')
coords['rating'] = np.where(coords['rating'] >= 10, coords['rating'] / 10, coords['rating'])

# Keep only needed columns
coords_clean = coords[[
    'Beach_Code',
    'Original_Beach_Name',
    'API_Beach_Name',
    'rating',
    'user_ratings_total',
    'lat',
    'lon'
]].copy()

# Filter valid Italian coordinates
coords_clean = coords_clean[
    (coords_clean['lat'].between(35, 48)) &
    (coords_clean['lon'].between(6, 20)) &
    coords_clean['lat'].notna() &
    coords_clean['lon'].notna()
]

print(f"\n Clean beaches with valid coordinates: {len(coords_clean)}")

if len(coords_clean) > 0:
    print("\nSample coordinates:")
    print(coords_clean[['Original_Beach_Name', 'lat', 'lon']].head(250))
    coords_clean.to_csv("beaches_final_clean.csv", index=False)
else:
    print("\n⚠️ Still 0 beaches. Let's check the data:")
    print(coords[['Original_Beach_Name', 'lat', 'lon']].head(10))
'''

Scaling coordinates...

 Clean beaches with valid coordinates: 60

Sample coordinates:
                                   Original_Beach_Name        lat        lon
0                                             Aciddara  38.080366  13.537345
1          Bagni della Regina Giovanna - Sorrento (Na)  40.633559  14.351224
3                          Boschetto Steccato di Cutro  38.934628  16.931957
4                                     Cala Santa Maria  38.708912  13.195334
5                                      Caletta Caretta  40.722365  17.771874
6                                 Caletta di Sant'Elia  38.096542  13.538822
10                            Castello di Santa Severa  42.016921  11.956440
11                                            Caulonia  38.337584  16.460348
12                                          Cipollazzo  37.573281  12.938365
14                   Club Adriatico - Polignano a Mare  41.020051  17.185052
15            da Borgo Bonsignore a Foce Fiume Platani  37.405059 